In [1]:
import azureml.core
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
import os
import shutil
import joblib
from azureml.train.automl import AutoMLConfig
from azureml.core import Model
from azureml.automl.core.shared import constants
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import Webservice, AciWebservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [2]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="AutoML-SDK")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-137235
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-137235


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# choose a name for your cluster
cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Compute target exists')
except ComputeTargetException:
    print('Creating a new compute target')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=5)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=10)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Creating a new compute target
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-04T15:05:54.393000+00:00', 'errors': None, 'creationTime': '2021-02-04T15:05:45.318544+00:00', 'modifiedTime': '2021-02-04T15:06:07.263606+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [4]:
ws.datasets.keys()

KeysView({'Houses': DatasetRegistration(id='dbd2435c-827f-4d02-8523-66c43b8d39f2', name='Houses', version=1, description='House prices and characteristics.', tags={})})

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Houses"
description_text = ""

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # azureml-core of version 1.0.72 or higher is required
        # azureml-dataprep[pandas] of version 1.1.34 or higher is required
        from azureml.core import Workspace, Dataset

        subscription_id = '9a7511b8-150f-4a58-8528-3e7d50216c31'
        resource_group = 'aml-quickstarts-137235'
        workspace_name = 'quick-starts-ws-137235'

        workspace = Workspace(subscription_id, resource_group, workspace_name)

        dataset = Dataset.get_by_name(workspace, name='Houses')
        dataset.to_pandas_dataframe()

df = dataset.to_pandas_dataframe()
df.describe()

,Transactieprijs_m2,BU_g_woz_v2018,BU_g_woz_v2016,WK_g_woz_v2019,BU_g_ink_po_v2017,Woonoppervlakte,BAGbouwjaar,maanden_sinds_jan2004,BUURT_m2_alle_objecten,Woningtype_appartement,...,Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345
count,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,51314.000000,5.131400e+04,51314.000000,...,50458.0,50774.0,50822.0,51314.000000,51314.000000,48512.0,51263.0,51202.0,51070.0,50961.0
mean,3545.260494,272.382644,220.076568,303.708949,35.010845,101.161009,1952.495245,109.128074,2.822468e+05,0.458627,...,-1.0,-1.0,-1.0,265.703654,277.547700,-1.0,-1.0,-1.0,-1.0,-1.0
std,1237.777275,94.270025,78.045218,66.238606,8.615431,46.006606,64.167687,60.577667,1.913169e+05,0.498290,...,0.0,0.0,0.0,1006.343176,951.008235,0.0,0.0,0.0,0.0,0.0
min,207.428571,-1.000000,-1.000000,177.000000,-1.000000,11.000000,1250.000000,0.000000,2.149800e+04,0.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
25%,2625.000000,212.000000,164.000000,250.000000,29.100000,74.000000,1926.000000,52.000000,1.827970e+05,0.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
50%,3294.117647,265.000000,214.000000,328.000000,33.900000,93.000000,1961.000000,119.000000,2.450660e+05,0.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
75%,4200.000000,326.000000,262.000000,354.000000,41.100000,120.000000,1997.000000,162.000000,3.260010e+05,1.000000,...,-1.0,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,-1.0
max,23890.909091,771.000000,647.000000,403.000000,63.800000,1145.000000,2020.000000,203.000000,1.649683e+06,1.000000,...,-1.0,-1.0,-1.0,5326.219181,4895.161290,-1.0,-1.0,-1.0,-1.0,-1.0


In [6]:
dataset.take(5).to_pandas_dataframe()

,Transactieprijs_m2,BU_g_woz_v2018,BU_g_woz_v2016,WK_g_woz_v2019,BU_g_ink_po_v2017,Woonoppervlakte,BAGbouwjaar,maanden_sinds_jan2004,BUURT_m2_alle_objecten,Woningtype_appartement,...,Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345,Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456,Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345,Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345
0,2700.000000,156,137,219,28.1,40,1962,1,290265,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
1,1725.000000,149,125,219,23.1,80,1956,1,413897,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
2,2350.000000,189,151,247,28.1,60,1979,1,184252,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
3,2875.000000,213,167,250,26.9,48,1919,1,116432,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1
4,2057.142857,156,137,219,28.1,70,1962,1,290265,1,...,-1,-1,-1,-1.0,-1.0,-1,-1,-1,-1,-1


In [7]:
#project_folder = './AutoML_SDK'

automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
    "primary_metric": 'spearman_correlation'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "regression",
                             training_data=dataset,
                             label_column_name="Transactieprijs_m2",   
                             #path = project_folder,
                             validation_size = 0.2,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
run = exp.submit(config=automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster
Parent Run ID: AutoML_18acecc2-d350-4355-b508-efc3585fffd3

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:  

In [9]:
RunDetails(run).show()
run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary de

{'runId': 'AutoML_18acecc2-d350-4355-b508-efc3585fffd3',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-04T15:07:23.356966Z',
 'endTimeUtc': '2021-02-04T15:39:04.482844Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'spearman_correlation',
  'train_split': '0.2',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoML-SDK","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group":"aml-quickstarts-137235","workspace_name":"quick-starts-ws-137235","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"spearman_correlation","task_type":"regression","data_script":null,"validation_size":0.2,"n_cross_validations":null,"y_min":null,"y_max":nul

In [10]:
# Retrieve and save your best automl model.
best_run, best_model = run.get_output()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [11]:
print(best_run)

Run(Experiment: AutoML-SDK,
Id: AutoML_18acecc2-d350-4355-b508-efc3585fffd3_38,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
print(best_model)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [13]:
best_run.get_tags()

{'_aml_system_azureml.automlComponent': 'AutoML',
 '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":1,"CurrentNodeCount":5}',
 'ensembled_iterations': '[0, 26, 15, 18, 5]',
 'ensembled_algorithms': "['LightGBM', 'GradientBoosting', 'XGBoostRegressor', 'XGBoostRegressor', 'XGBoostRegressor']",
 'ensemble_weights': '[0.4, 0.4, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667]',
 'best_individual_pipeline_score': '0.9366140595282841',
 'best_individual_iteration': '0',
 '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}

In [14]:
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

normalized_mean_absolute_error 0.01296623278104177
root_mean_squared_error 456.28935124613594
median_absolute_error 211.9121639257296
root_mean_squared_log_error 0.1147484893262926
spearman_correlation 0.9382001490115334
explained_variance 0.869123976639181
normalized_median_absolute_error 0.00894767826677435
r2_score 0.8677437129434453
mean_absolute_error 307.0855214808526
normalized_root_mean_squared_error 0.019266144216886587
mean_absolute_percentage_error 8.534838836311103
normalized_root_mean_squared_log_error 0.024199869158998927
residuals aml://artifactId/ExperimentRun/dcid.AutoML_18acecc2-d350-4355-b508-efc3585fffd3_38/residuals
predicted_true aml://artifactId/ExperimentRun/dcid.AutoML_18acecc2-d350-4355-b508-efc3585fffd3_38/predicted_true


In [15]:
#best_run.register_model(model_name = "automl_udacity_project.pkl", model_path = './outputs/')
print(best_model._final_estimator)

PreFittedSoftVotingRegressor(estimators=[('0',
                                          Pipeline(memory=None,
                                                   steps=[('maxabsscaler',
                                                           MaxAbsScaler(copy=True)),
                                                          ('lightgbmregressor',
                                                           LightGBMRegressor(boosting_type='gbdt',
                                                                             class_weight=None,
                                                                             colsample_bytree=1.0,
                                                                             importance_type='split',
                                                                             learning_rate=0.1,
                                                                             max_depth=-1,
                                                                  

In [16]:
best_run.register_model(model_path='outputs/model.pkl', model_name='automl_model',
                        tags={'Training context':'AutoML'},
                        properties={'Spearman correleation': best_run_metrics['spearman_correlation'], 'MAE': best_run_metrics['mean_absolute_error']})

Model(workspace=Workspace.create(name='quick-starts-ws-137235', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-137235'), name=automl_model, id=automl_model:1, version=1, tags={'Training context': 'AutoML'}, properties={'Spearman correleation': '0.9382001490115334', 'MAE': '307.0855214808526'})

In [17]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

automl_model version: 1
	 Training context : AutoML
	 Spearman correleation : 0.9382001490115334
	 MAE : 307.0855214808526




In [18]:
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

In [37]:
inference_config = InferenceConfig(entry_script="score.py")

In [38]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)

In [40]:
service=Model.deploy(workspace=ws,
                    name="houses-service2",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [41]:
print(service.get_logs())

2021-02-04T16:33:22,952492000+00:00 - iot-server/run 
2021-02-04T16:33:22,955294400+00:00 - gunicorn/run 
2021-02-04T16:33:22,963190000+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-04T16:33:22,980963500+00:00 - rsyslog/run 
rsyslogd

In [42]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)


service state: Healthy

scoring URI: 
http://1f4fb9d2-03bb-4aaa-ba70-26b83fe2ae51.southcentralus.azurecontainer.io/score

swagger URI: 
http://1f4fb9d2-03bb-4aaa-ba70-26b83fe2ae51.southcentralus.azurecontainer.io/swagger.json

http://1f4fb9d2-03bb-4aaa-ba70-26b83fe2ae51.southcentralus.azurecontainer.io/score
http://1f4fb9d2-03bb-4aaa-ba70-26b83fe2ae51.southcentralus.azurecontainer.io/swagger.json


In [46]:
import requests
import json

data={"data":
  [{"BU_g_woz_v2018": 156,
    "BU_g_woz_v2016": 137,
    "WK_g_woz_v2019": 219,
    "BU_g_ink_po_v2017": 28.1,
    "Woonoppervlakte": 40,
    "BAGbouwjaar": 1962,
    "maanden_sinds_jan2004": 1,
    "BUURT_m2_alle_objecten": 290265,
    "Woningtype_appartement": 1,
    "BAG_perceeloppervlakte": 3294,
    "Woningtype_vrijstaand": 0,
    "Woningtype_tussenwoning": 0,
    "BU_ste_oad_v2018": 3475,
    "WK_ste_oad_v2018": 3460,
    "Hoofdweg_Distance": 4589,
    "hoogte": 17.17,
    "Gemeentehuis_Distance": 2158,
    "Trein_Distance": 1308,
    "Park_Distance": 1196,
    "WIJK_aantal_objecten": 21470,
    "Autosnelweg_Distance": 1468,
    "BUURT_median_bouwjaar_appartementen": 1968,
    "Koop_historisch_2020_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2019_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2018_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2017_med_transactieprijsm2_PC_123456": 2553.3126,
    "Koop_historisch_2016_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2015_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2014_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2013_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2012_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2011_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2010_med_transactieprijsm2_PC_123456": -1,
    "Koop_historisch_2020_med_transactieprijsm2_PC_12345": 3225.4902,
    "Koop_historisch_2019_med_transactieprijsm2_PC_12345": 3256.1728,
    "Koop_historisch_2018_med_transactieprijsm2_PC_12345": 3132.3529,
    "Koop_historisch_2017_med_transactieprijsm2_PC_12345": 2628.5714,
    "Koop_historisch_2016_med_transactieprijsm2_PC_12345": 1957.1429,
    "Koop_historisch_2015_med_transactieprijsm2_PC_12345": 1816.5631,
    "Koop_historisch_2014_med_transactieprijsm2_PC_12345": 1695.0704,
    "Koop_historisch_2013_med_transactieprijsm2_PC_12345": 1739.0873,
    "Koop_historisch_2012_med_transactieprijsm2_PC_12345": 1620,
    "Koop_historisch_2011_med_transactieprijsm2_PC_12345": 2114.2857,
    "Koop_historisch_2010_med_transactieprijsm2_PC_12345": 1966.6667,
    "Koop_historisch_2020_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2019_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2018_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2017_med_transactieprijsm2_appartementen_PC_123456": 2553.3126,
    "Koop_historisch_2016_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2015_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2014_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2013_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2012_med_transactieprijsm2_appartementen_PC_123456": -1,
    "Koop_historisch_2020_med_transactieprijsm2_appartementen_PC_12345": 3225.4902,
    "Koop_historisch_2019_med_transactieprijsm2_appartementen_PC_12345": 3256.1728,
    "Koop_historisch_2018_med_transactieprijsm2_appartementen_PC_12345": 3132.3529,
    "Koop_historisch_2017_med_transactieprijsm2_appartementen_PC_12345": 2628.5714,
    "Koop_historisch_2016_med_transactieprijsm2_appartementen_PC_12345": 1957.1429,
    "Koop_historisch_2015_med_transactieprijsm2_appartementen_PC_12345": 1816.5631,
    "Koop_historisch_2014_med_transactieprijsm2_appartementen_PC_12345": 1700,
    "Koop_historisch_2013_med_transactieprijsm2_appartementen_PC_12345": 1739.0873,
    "Koop_historisch_2012_med_transactieprijsm2_appartementen_PC_12345": 1620,
    "Koop_historisch_2020_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
    "Koop_historisch_2019_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
    "Koop_historisch_2018_med_transactieprijsm2_vrijstaanden_PC_12345": -1,
    "Koop_historisch_2019_med_transactieprijsm2_vrijstaanden_PC_123456": -1,
    "Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
    "Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
    "Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_12345": -1,
    "Koop_historisch_2020_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
    "Koop_historisch_2019_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
    "Koop_historisch_2018_med_transactieprijsm2_tussenwoningen_PC_123456": -1,
    "Koop_historisch_2020_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
    "Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
    "Koop_historisch_2018_med_transactieprijsm2_hoekwoningen_PC_12345": -1,
    "Koop_historisch_2019_med_transactieprijsm2_hoekwoningen_PC_123456": -1,
    "Koop_historisch_2020_med_transactieprijsm2_2onder1kappers_PC_12345": -1,
    "Koop_historisch_2019_med_transactieprijsm2_2onder1kappers_PC_12345": -1,
    "Koop_historisch_2018_med_transactieprijsm2_2onder1kappers_PC_12345": -1}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("The output is: ",result.json())

The output is:  [2954.471794267492]


In [48]:
result.status_code

200

In [49]:
print(scoring_uri)

http://1f4fb9d2-03bb-4aaa-ba70-26b83fe2ae51.southcentralus.azurecontainer.io/score


In [ ]:
#service.delete()
#compute_target.delete()
#model.delete()
#run.delete()
#exp.delete()